In [1]:
import csv
import sqlite3
import functions
import pandas as pd
import requests as rq
from config import api_key
# pip install geopy
import time
from math import ceil
import pickle

In [2]:
df = pickle.load(open('df.pkl','rb'))

In [3]:
df

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Title Polarity,Title Subjectivity,Class_Business Class,Class_Economy Class,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,0.600,0.9,True,False,False,False,False,False,False,True
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,-1.000,1.0,False,True,False,False,False,False,False,True
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,0.000,0.0,False,True,False,False,False,False,True,False
3,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.300,0.3,False,True,False,False,False,False,False,True
4,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.700,0.6,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,Korean Air,6,2016,5,4,5,3,4,7,1,...,0.000,0.0,False,True,False,False,True,False,False,False
8096,Korean Air,6,2016,3,5,5,4,5,10,1,...,0.000,0.0,False,True,False,False,False,True,False,False
8097,Korean Air,4,2016,4,5,5,5,1,2,0,...,0.000,0.0,True,False,False,False,True,False,False,False
8098,Korean Air,4,2016,5,1,3,4,5,10,1,...,0.200,0.1,True,False,False,False,True,False,False,False


In [4]:
print(f'Rows and Columns: {df.shape}')
df.columns

Rows and Columns: (7683, 31)


Index(['Airline', 'Month Flown', 'Year Flown', 'Seat Comfort', 'Staff Service',
       'Food & Beverages', 'Inflight Entertainment', 'Value For Money',
       'Overall Rating', 'Recommended', 'Origin', 'Destination', 'Via',
       'Origin_Code', 'Destination_Code', 'Via_Code', 'Origin_City',
       'Destination_City', 'Via_City', 'Comment Polarity',
       'Comment Subjectivity', 'Title Polarity', 'Title Subjectivity',
       'Class_Business Class', 'Class_Economy Class', 'Class_First Class',
       'Class_Premium Economy', 'Type of Traveller_Business',
       'Type of Traveller_Couple Leisure', 'Type of Traveller_Family Leisure',
       'Type of Traveller_Solo Leisure'],
      dtype='object')

In [7]:
values_to_drop = ['LED', 'KBL', 'IST', 'VLC', 'RZV', 'LON', 'CBU', 'SKG', 'NYC']
columns_to_check = ['Origin_Code', 'Destination_Code', 'Via_Code']

# Create a boolean mask to identify rows with values to drop
mask = df[columns_to_check].isin(values_to_drop).any(axis=1)

# Drop rows that contain the specified values
df = df[~mask]

# Display the cleaned DataFrame
df

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Title Polarity,Title Subjectivity,Class_Business Class,Class_Economy Class,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,0.600,0.9,True,False,False,False,False,False,False,True
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,-1.000,1.0,False,True,False,False,False,False,False,True
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,0.000,0.0,False,True,False,False,False,False,True,False
3,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.300,0.3,False,True,False,False,False,False,False,True
4,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.700,0.6,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,Korean Air,6,2016,5,4,5,3,4,7,1,...,0.000,0.0,False,True,False,False,True,False,False,False
8096,Korean Air,6,2016,3,5,5,4,5,10,1,...,0.000,0.0,False,True,False,False,False,True,False,False
8097,Korean Air,4,2016,4,5,5,5,1,2,0,...,0.000,0.0,True,False,False,False,True,False,False,False
8098,Korean Air,4,2016,5,1,3,4,5,10,1,...,0.200,0.1,True,False,False,False,True,False,False,False


In [13]:
df['Destination_Code'].nunique()

292

In [14]:
unique_codes_Origin = df['Origin_Code'].unique()
len(unique_codes_Origin)
unique_codes_Origin

array(['CGK', 'HKT', 'SAI', 'SIN', 'BWN', 'LHR', 'PER', 'MIL', 'BKK',
       'KUL', 'PEN', 'FRA', 'BNE', 'MEL', 'AKL', 'JNB', 'SYD', 'MNL',
       'BOM', 'ATH', 'CAN', 'KCZ', 'DXB', 'SFO', 'ADL', 'CPT', 'EWR',
       'HKG', 'YVR', 'PNH', 'DPS', 'HAN', 'MAN', 'JFK', 'LAX', 'SUB',
       'PAR', 'BLR', 'AMS', 'SGN', 'DAD', 'ROM', 'CCU', 'MUC', 'CMB',
       'SAO', 'KIX', 'HND', 'ZRH', 'UPG', 'AMD', 'CPH', 'PUS', 'SPK',
       'PVG', 'DEL', 'DAC', 'SEA', 'ARN', 'CHC', 'ICN', 'DUS', 'NRT',
       'WLG', 'BCN', 'TPE', 'RGN', 'DVO', 'CJB', 'MAA', 'IAH', 'CBR',
       'DME', 'MXP', 'CDG', 'FCO', 'DOH', 'ORD', 'NCE', 'TBS', 'BEY',
       'LGW', 'DFW', 'KWI', 'LUN', 'BRU', 'RUH', 'OSL', 'KTM', 'VIE',
       'BWI', 'AUH', 'EBB', 'PHL', 'WAW', 'EDI', 'GVA', 'YMQ', 'DAL',
       'MPM', 'BUH', 'DUB', 'BOS', 'MAD', 'IKA', 'SOF', 'LOS', 'PRG',
       'NBO', 'ATL', 'ALG', 'SHJ', 'MIA', 'ZAG', 'COK', 'ISB', 'ESB',
       'KGL', 'BSR', 'MLE', 'YTO', 'TUN', 'CAI', 'BJV', 'AGP', 'CTU',
       'BEG', 'DMM',

In [15]:
unique_codes_Dest = df['Destination_Code'].unique()
len(unique_codes_Dest)
unique_codes_Dest

array(['SIN', 'LHR', 'PNH', 'CNS', 'FRA', 'BWN', 'CGK', 'HND', 'HKG',
       'BKK', 'MEL', 'ADL', 'MAN', 'SYD', 'AKL', 'PER', 'PEN', 'ZAG',
       'DPS', 'LAX', 'JFK', 'DXB', 'HKT', 'FUK', 'CHC', 'PAR', 'DEL',
       'ZRH', 'KIX', 'SGN', 'MIL', 'SEA', 'JNB', 'KUL', 'MNL', 'BNE',
       'CPH', 'IAH', 'SFO', 'YTO', 'AMS', 'CPT', 'BOM', 'MUC', 'CCU',
       'SAI', 'EWR', 'MLE', 'SVO', 'DUB', 'NRT', 'TPE', 'NGO', 'USM',
       'UPG', 'CMB', 'ICN', 'HAN', 'CJB', 'TAO', 'PVG', 'TRV', 'BLR',
       'KCZ', 'CBR', 'SUB', 'PEK', 'ATH', 'ROM', 'WLG', 'DUS', 'DAC',
       'ARN', 'XMN', 'AMD', 'CAN', 'BCN', 'MAA', 'VTE', 'CDG', 'CNX',
       'DME', 'GRU', 'RGN', 'MXP', 'YMQ', 'DOH', 'JRO', 'DET', 'BSR',
       'GOI', 'BRU', 'WAW', 'SLL', 'LYS', 'BWI', 'LUN', 'KWI', 'SAO',
       'ORD', 'LGW', 'KTM', 'MCT', 'VIE', 'BAH', 'AUH', 'EDI', 'IKA',
       'BER', 'WDH', 'PHL', 'RUH', 'BGW', 'LOS', 'BUH', 'DAL', 'MAD',
       'NBO', 'SHJ', 'ISB', 'PRG', 'OSL', 'BOS', 'DMM', 'LHE', 'MIA',
       'CEB', 'CAI',

In [17]:
unique_codes_Origin = set(unique_codes_Origin)
unique_codes_Dest = set(unique_codes_Dest)
unique_codes = unique_codes_Origin.union(unique_codes_Dest)
unique_codes

{'ABJ',
 'ABV',
 'ABZ',
 'ADD',
 'ADL',
 'AEX',
 'AGP',
 'AJA',
 'AKL',
 'ALA',
 'ALG',
 'AMD',
 'AMM',
 'AMS',
 'AOJ',
 'ARN',
 'ATH',
 'ATL',
 'ATQ',
 'AUH',
 'AUS',
 'AXT',
 'AYT',
 'BAH',
 'BAK',
 'BCN',
 'BEG',
 'BER',
 'BEY',
 'BGO',
 'BGW',
 'BHM',
 'BHX',
 'BIA',
 'BIO',
 'BIQ',
 'BJV',
 'BKK',
 'BKO',
 'BLQ',
 'BLR',
 'BNE',
 'BOD',
 'BOG',
 'BOM',
 'BOS',
 'BRE',
 'BRU',
 'BSL',
 'BSR',
 'BUD',
 'BUE',
 'BUH',
 'BWI',
 'BWN',
 'BXH',
 'CAI',
 'CAN',
 'CAS',
 'CAY',
 'CBR',
 'CCJ',
 'CCU',
 'CDG',
 'CEB',
 'CGK',
 'CGN',
 'CHC',
 'CJB',
 'CJU',
 'CKG',
 'CKY',
 'CLJ',
 'CMB',
 'CMN',
 'CNS',
 'CNX',
 'COK',
 'CPH',
 'CPT',
 'CRK',
 'CTA',
 'CTS',
 'CTU',
 'CUN',
 'CVG',
 'CWL',
 'DAC',
 'DAD',
 'DAL',
 'DAR',
 'DBV',
 'DEL',
 'DEN',
 'DET',
 'DFW',
 'DKR',
 'DLA',
 'DLC',
 'DLM',
 'DME',
 'DMM',
 'DOH',
 'DPS',
 'DTW',
 'DUB',
 'DUR',
 'DUS',
 'DVO',
 'DWC',
 'DXB',
 'DYU',
 'EBB',
 'ECN',
 'EDI',
 'ESB',
 'EVN',
 'EWR',
 'EZE',
 'FCO',
 'FDF',
 'FLL',
 'FLR',
 'FNA',
 'FOR',


In [20]:
len(unique_codes)

357

In [21]:
pickle.dump(df, open('df_2.pkl','wb'))